In [6]:
import pandas as pd
import numpy as np
import os, re

In [7]:
files = [i for i in os.listdir('.') if i.endswith('xlsx')]

In [8]:
def replaceColumnLabel(df, row = 0, fillna = ''):
  col = df.iloc[row].fillna(fillna)
  df = df.iloc[row+1:]
  df.columns = col
  return df

In [15]:
def process(path):
  year = re.findall('\d+', path)[0]
  df = pd.read_excel(path, skiprows=1)
  international = df.loc[:,'Keberangkatan pada Penerbangan Internasional':]
  international = replaceColumnLabel(international, row=1, fillna='airport').dropna()
  international = np.ravel(international)
  international = np.delete(international, np.where(international == '-'))
  international_sum = np.sum(international.astype(int))
  national = df.loc[:, 'Keberangkatan pada Penerbangan Domestik':'Keberangkatan pada Penerbangan Internasional'].drop('Keberangkatan pada Penerbangan Internasional', axis=1)
  national = replaceColumnLabel(national, row=1, fillna='airport').dropna()
  national = np.ravel(national)
  national = np.delete(national, np.where(national == '-'))
  national_sum = np.sum(national.astype(int))
  res_df = pd.DataFrame({'tahun': [year, year], 'Jumlah': [national_sum, international_sum], 'penerbangan':['nasional', 'internasional']})
  return res_df

In [16]:
dfs = [process(i) for i in files]

In [17]:
df = pd.concat(dfs).sort_values('tahun')
df

,tahun,Jumlah,penerbangan
0,2019,36494088,nasional
1,2019,17108874,internasional
0,2020,32511290,nasional
1,2020,6520358,internasional
0,2021,8759617,nasional
1,2021,356016,internasional


In [18]:
df.to_csv('total keberangkatan melalui airport 2019-2020.csv', index=False)